![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/main/extensions/resource-deployment/images/microsoft-small-logo.png)

## Run SQL Server 2022 container image with Docker

This notebook will use Docker to pull and run the SQL Server 2022 container image and connect to it in Azure Data Studio

### Dependencies

- Docker Engine. For more information, see [Install Docker](https://docs.docker.com/engine/installation/).

<span style="color:red;"><font size="3">Please press the "Run all" button to run the notebook</font></span>

### Check dependencies

In [6]:
import sys,os,getpass,json,html,time
from string import Template

def run_command(displayCommand = ""):
    print("Executing: " + displayCommand if displayCommand != "" else cmd)
    !{cmd}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n')
    print(f'Command successfully executed')

cmd = 'docker version'
run_command()

docker version
Client:
 Version:           20.10.23
 API version:       1.41
 Go version:        go1.19.5
 Git commit:        715524332f
 Built:             Sat Jan 21 15:13:25 2023
 OS/Arch:           linux/amd64
 Context:           default
 Experimental:      true

Server:
 Engine:
  Version:          20.10.23
  API version:      1.41 (minimum version 1.12)
  Go version:       go1.19.5
  Git commit:       6051f14291
  Built:            Sat Jan 21 15:12:19 2023
  OS/Arch:          linux/amd64
  Experimental:     false
 containerd:
  Version:          v1.6.16
  GitCommit:        31aa4358a36870b21a992d3ad2bef29e1d693bec.m
 runc:
  Version:          1.1.4
  GitCommit:        
 docker-init:
  Version:          0.19.0
  GitCommit:        de40ad0


Command successfully executed


### List existing containers
You can view the ports that have been used by existing containers

In [7]:
cmd = f'docker ps -a'
run_command()

docker ps -a


CONTAINER ID   IMAGE                                        COMMAND                  CREATED          STATUS                        PORTS     NAMES
2fd6f2c4a341   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/perm…"   8 minutes ago    Exited (255) 4 minutes ago              friendly_black
93ee26e9d4fb   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/perm…"   17 minutes ago   Exited (255) 10 minutes ago             stoic_kalam
2516b731ac7d   test                                         "/bin/sh -c 'cd Simp…"   40 hours ago     Exited (0) 40 hours ago                 nifty_wilson
16184d9f2002   74bf5cbe6bcf                                 "/bin/sh -c 'cd Simp…"   40 hours ago     Exited (0) 40 hours ago                 recursing_ride
ecdd6d710aea   9558641a20d9                                 "/bin/sh -c 'cd Simp…"   40 hours ago     Exited (0) 40 hours ago                 objective_lovelace
364549dcb795   9558641a20d9                                 "/bin/sh

Command successfully executed


### Required information

In [8]:
env_var_flag = "AZDATA_NB_VAR_DOCKER_PASSWORD" in os.environ
password_name = 'SQL Server sa account password'
if env_var_flag:
    sql_password = os.environ["AZDATA_NB_VAR_DOCKER_PASSWORD"]
    sql_port = os.environ["AZDATA_NB_VAR_DOCKER_PORT"]
else:
    sql_password = getpass.getpass(prompt = password_name)
    password_confirm = getpass.getpass(prompt = f'Confirm {password_name}')
    if sql_password != password_confirm:
        raise SystemExit(f'{password_name} does not match the confirmation password.')
    sql_port = input('SQL Server port, default value is 1433')
    if len(sql_port) == 0:
        sql_port = '1433'
print(f'{password_name}: ******')
print(f'Port: {sql_port}')

SQL Server sa account password: ******
Port: 1433


### Pull the container image

In [9]:
cmd = f'docker pull mcr.microsoft.com/mssql/server:2022-latest'
run_command()

docker pull mcr.microsoft.com/mssql/server:2022-latest


2022-latest: Pulling from mssql/server
Digest: sha256:25171ecc78127fe53c9c12506a40577de8bef955a8b6a4bf9b8757c4c4601182
Status: Image is up to date for mcr.microsoft.com/mssql/server:2022-latest
mcr.microsoft.com/mssql/server:2022-latest


Command successfully executed


### Start a new container

In [10]:
if env_var_flag:
    container_name = os.environ["AZDATA_NB_VAR_DOCKER_CONTAINER_NAME"]
else:
    container_name = 'sql2022-' + time.strftime("%Y%m%d%H%M%S", time.localtime())
print('New container name: ' + container_name)

template = Template(f'docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD=$password" -p {sql_port}:1433 --name {container_name} -d mcr.microsoft.com/mssql/server:2022-latest')
cmd = template.substitute(password=sql_password)
run_command(template.substitute(password='******'))

New container name: sql2022-20230303111647
Executing: docker run -e ACCEPT_EULA=Y -e "SA_PASSWORD=******" -p 1433:1433 --name sql2022-20230303111647 -d mcr.microsoft.com/mssql/server:2022-latest


fa0b27d494489a552629e8f009860048dd3889c8add52e29125801f0438c0dfb


Command successfully executed


### List all the containers

In [11]:
cmd = f'docker ps -a'
run_command()

docker ps -a


CONTAINER ID   IMAGE                                        COMMAND                  CREATED          STATUS                        PORTS                                       NAMES
fa0b27d49448   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/perm…"   1 second ago     Up Less than a second         0.0.0.0:1433->1433/tcp, :::1433->1433/tcp   sql2022-20230303111647
2fd6f2c4a341   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/perm…"   8 minutes ago    Exited (255) 4 minutes ago                                                friendly_black
93ee26e9d4fb   mcr.microsoft.com/mssql/server:2022-latest   "/opt/mssql/bin/perm…"   17 minutes ago   Exited (255) 10 minutes ago                                               stoic_kalam
2516b731ac7d   test                                         "/bin/sh -c 'cd Simp…"   40 hours ago     Exited (0) 40 hours ago                                                   nifty_wilson
16184d9f2002   74bf5cbe6bcf                        

Command successfully executed


### Connect to SQL Server in Azure Data Studio
It might take a couple minutes for SQL Server to launch

In [12]:
from IPython.display import *
connectionParameter = '{"serverName":"localhost,' + sql_port + '","providerName":"MSSQL","authenticationType":"SqlLogin","userName":"sa","password":' + json.dumps(sql_password) + ',"options": {"trustServerCertificate":true}}'
display(HTML('<br/><a href="command:azdata.connect?' + html.escape(connectionParameter)+'"><font size="3">Click here to connect to SQL Server</font></a><br/>'))
display(HTML('<br/><span style="color:red"><font size="2">NOTE: The SQL Server password is included in this link, you may want to clear the results of this code cell before saving the notebook.</font></span>'))

### Stop and remove the container

In [13]:
stop_container_command = f'docker stop {container_name}'
remove_container_command = f'docker rm {container_name}'
display(HTML("Use this link to: <a href=\"command:workbench.action.terminal.focus\">open the terminal window in Azure Data Studio</a> and use the links below to paste the command to the terminal."))
display(HTML("Stop the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+stop_container_command.replace(" ","%20")+"%22%7D\">" + stop_container_command + "</a>"))
display(HTML("Remove the container: <a href=\"command:workbench.action.terminal.sendSequence?%7B%22text%22%3A%22"+remove_container_command.replace(" ","%20")+"%22%7D\">" + remove_container_command + "</a>"))